# Modeling – Heart Disease Prediction

Ce notebook vise à entraîner et comparer plusieurs modèles de classification
afin de prédire la présence d’une maladie cardiaque à partir des données cliniques.
Les modèles sont évalués à l’aide de métriques adaptées.


In [1]:
import pandas as pd

In [2]:
from pathlib import Path

DATA_PATH = Path("..") / "dataset.csv"
df = pd.read_csv(DATA_PATH)

# Suppression de l'identifiant
df = df.drop(columns=["id"])

df.head()


,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0


## Chargement des données et séparation train/test

Les données sont rechargées et séparées en jeux d’entraînement et de test
en utilisant les mêmes paramètres que lors de la phase de preprocessing,
afin d’assurer la cohérence des résultats.


In [3]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=["num"])
y = df["num"]

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

X_train.shape, X_test.shape


((736, 14), (184, 14))

## Pipeline de preprocessing

Un pipeline de preprocessing est mis en place afin de gérer :
- l’imputation des valeurs manquantes,
- la standardisation des variables numériques,
- l’encodage des variables catégorielles.

Ce pipeline est intégré directement aux modèles afin d’éviter toute fuite de données.


In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

num_features = X_train.select_dtypes(include=["int64", "float64"]).columns
cat_features = X_train.select_dtypes(include=["object"]).columns

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_features),
        ("cat", categorical_transformer, cat_features)
    ]
)


## Modèle 1 – Régression logistique (baseline)

La régression logistique est utilisée comme modèle de référence en raison de sa
simplicité et de son interprétabilité. Elle permet d’évaluer rapidement si les
variables contiennent un signal prédictif pertinent.


In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

log_reg = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("model", LogisticRegression(max_iter=1000))
])

log_reg.fit(X_train, y_train)

y_pred_lr = log_reg.predict(X_test)

acc_lr = accuracy_score(y_test, y_pred_lr)
f1_lr = f1_score(y_test, y_pred_lr, average="weighted")

acc_lr, f1_lr



(0.5869565217391305, 0.5614639051891325)

## Modèle 2 – Random Forest

Les Random Forest sont des modèles ensemblistes capables de modéliser des relations
non linéaires entre les variables. Ils sont généralement robustes et performants
sur des données tabulaires.



In [6]:
from sklearn.ensemble import RandomForestClassifier

rf = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("model", RandomForestClassifier(
        n_estimators=200,
        random_state=42
    ))
])

rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)

acc_rf = accuracy_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf, average="weighted")

acc_rf, f1_rf


(0.5869565217391305, 0.5687354137080654)

## Modèle 3 – Gradient Boosting

Le Gradient Boosting combine plusieurs modèles faibles afin d’améliorer
progressivement les performances. Il est souvent très performant pour les
problèmes de classification structurés.


In [7]:
from sklearn.ensemble import GradientBoostingClassifier

gb = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("model", GradientBoostingClassifier(random_state=42))
])

gb.fit(X_train, y_train)

y_pred_gb = gb.predict(X_test)

acc_gb = accuracy_score(y_test, y_pred_gb)
f1_gb = f1_score(y_test, y_pred_gb, average="weighted")

acc_gb, f1_gb


(0.6141304347826086, 0.59259280658674)

## Comparaison des performances

Les modèles sont comparés à l’aide de l’accuracy et du F1-score pondéré.


In [8]:
results = pd.DataFrame({
    "Model": [
        "Logistic Regression",
        "Random Forest",
        "Gradient Boosting"
    ],
    "Accuracy": [
        acc_lr,
        acc_rf,
        acc_gb
    ],
    "F1-score": [
        f1_lr,
        f1_rf,
        f1_gb
    ]
})

results


,Model,Accuracy,F1-score
0,Logistic Regression,0.586957,0.561464
1,Random Forest,0.586957,0.568735
2,Gradient Boosting,0.614130,0.592593


## Conclusion

La régression logistique sert de modèle de référence.  
Les modèles ensemblistes obtiennent de meilleures performances, indiquant la présence de relations non linéaires dans les données.  
Le Gradient Boosting présente les meilleurs résultats globaux et sera retenu pour la phase d’optimisation des hyperparamètres.
